In [ ]:
!pip install blitz-bayesian-pytorch
!pip install -e git+git://github.com/vonHacht/yfinance.git@master#egg=yfinance
#!pip install optuna
!pip install --upgrade ta
#!pip install pandas_ta
!pip install -U git+https://github.com/twopirllc/pandas-ta
!pip install git+git://github.com/peerchemist/finta.git
!pip install stumpy
!pip install tslearn
!pip install impyute
!pip install -U scikit-learn
!pip install fbprophet
 
 
!pip install quandl
!pip install pytrends
!pip install fracdiff
!pip install PyPortfolioOpt

     |████████████████████████████████| 51kB 5.6MB/s 
Obtaining yfinance from git+git://github.com/vonHacht/yfinance.git@master#egg=yfinance
  Cloning git://github.com/vonHacht/yfinance.git (to revision master) to ./src/yfinance
  Running command git clone -q git://github.com/vonHacht/yfinance.git /content/src/yfinance
  Running setup.py develop for yfinance
  Created wheel for ta: filename=ta-0.7.0-cp37-none-any.whl size=28716 sha256=e27b4c13d04c7c0c6fbabc642c93c787a10622d2a797a6588afa08d2b6f18f7c
  Stored in directory: /root/.cache/pip/wheels/dd/88/30/de9553fb54a474eb7480b937cdbb140bdda613d29cf4da7994
Successfully built ta
  Cloning https://github.com/twopirllc/pandas-ta to /tmp/pip-req-build-dm_i4wts
  Running command git clone -q https://github.com/twopirllc/pandas-ta /tmp/pip-req-build-dm_i4wts
  Created wheel for pandas-ta: filename=pandas_ta-0.2.75b0-cp37-none-any.whl size=197080 sha256=f124a72d0a9728331200b88de466c6afcf2bf84fafd2f0bb6f7b36fbc8dcc3c5
  Stored in directory: /tmp/

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
#import optuna
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import quandl
import blitz
from blitz.modules import BayesianLSTM
from blitz.modules import BayesianLinear
from blitz.utils import variational_estimator
import yfinance as yf
import pandas_datareader as pdr
import ta
from pandas import DataFrame, Series
from finta import TA
from scipy.stats import boxcox
import stumpy
import tslearn
from sklearn import metrics
import math
from google.colab import files
import csv
import inspect
import impyute
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from fracdiff import FracdiffStat,fdiff
from tslearn.utils import to_time_series_dataset
from tslearn.svm import TimeSeriesSVC
from tslearn.neural_network import TimeSeriesMLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.metrics import f1_score as f1
from sklearn.linear_model import SGDClassifier as SGC
import lightgbm as lgb
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import IsolationForest
from fbprophet import Prophet
from scipy import signal
import pytrends
from pytrends import dailydata
from datetime import date, timedelta
from functools import partial
from time import sleep
from pytrends.exceptions import ResponseError
from pytrends.request import TrendReq
from sklearn.ensemble import AdaBoostClassifier,BaggingClassifier
from datetime import datetime
from sklearn.inspection import permutation_importance as pimp
import pypfopt
from sklearn.neighbors import BallTree
from sklearn.neighbors import DistanceMetric
from scipy.stats import mode
 
import warnings
warnings.simplefilter("ignore")
 
from google.colab import drive
drive.mount('drive')

####Pick Device

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device) 

In [ ]:
def Mult_Ticker_GTrend():
  string=input()
  ticker_list=string.split(',')
  collector=[]
    
  for ticker in ticker_list:
    ticker_df=GTrends_Daily(ticker + ' stock')
    collector.append(ticker_df)
  
  tickers_df=pd.concat(collector,axis=1)
  

  return tickers_df

In [ ]:
def DropDate(klepto_df):
  klepto_df.index=pd.to_datetime(klepto_df['date'])
  
  klepto_df=klepto_df.drop('date',axis=1)
  
  return klepto_df

In [ ]:
def Add_Gtrends():
  string=input()
  term_list=string.split(',')
  to_add_list=[GTrends_Daily(term) for term in term_list]
  main_gtrends_df=UpdateUniGTrends()
  to_add_list.append(main_gtrends_df)

  main_gtrends_df=pd.concat(to_add_list,axis=1)

  main_gtrends_df.dropna().to_csv('drive/My Drive/Nick Fury/External Datasets/gtrends.csv/gtrends.csv')
  main_gtrends_df=UpdateUniGTrends()
  
  return main_gtrends_df

In [ ]:
def UpdateTickerGTrends(ticker):
  ticker_gtrends_df=pd.read_csv('drive/My Drive/Nick Fury/External Datasets/gtrends.csv/{}.csv'.format(ticker))

  ticker_gtrends_df.index=ticker_gtrends_df.date
  ticker_gtrends_df=ticker_gtrends_df.iloc[:,1:]
  ticker_gtrends_df.index=pd.to_datetime(ticker_gtrends_df.index)

  timeframe='now 7-d'

  keyword=ticker+' stock'

  pytrends_object = pytrends.request.TrendReq(hl='en-US', tz=360)

  pytrends_object.build_payload([keyword],timeframe=timeframe)

  single_gtrends_df=pytrends_object.interest_over_time()[keyword]

  single_gtrends_df=single_gtrends_df.resample('1D').mean()

  ticker_gtrends_df=pd.concat([ticker_gtrends_df,pd.DataFrame(single_gtrends_df)])

  ticker_gtrends_df = ticker_gtrends_df[~ticker_gtrends_df.index.duplicated(keep='first')]

  ticker_gtrends_df.to_csv('drive/My Drive/Nick Fury/External Datasets/gtrends.csv/{}.csv'.format(ticker))

  return ticker_gtrends_df

#Data

####External Data

In [ ]:
def GTrends_Daily(search_term):
  daily_data=pytrends.dailydata.get_daily_data(search_term,start_year=2010,start_mon=1,stop_year=datetime.now().date().year,stop_mon=datetime.now().date().month)

  data=daily_data[search_term+'_unscaled']
  data=data.rename(search_term)

  return data

def Add_Weekend(main_gtrends_df):
  weekend_df=main_gtrends_df[main_gtrends_df.index.dayofweek>4]
  weekend_df.index=pd.to_datetime(weekend_df.index)

  collector=list()
  for idx in range(len(weekend_df)-1):
    if weekend_df.iloc[idx+1].name.date()-weekend_df.iloc[idx].name.date()==timedelta(days=1):
      collector.append((weekend_df.iloc[idx+1]+weekend_df.iloc[idx]))

  average_df=pd.concat(collector,axis=1).T
  average_df.index=weekend_df.index[::2]
  average_df.columns=average_df.columns+'_wknd'

  average_df['week']=0
  for idx in range(len(average_df)):
    average_df['week'].iloc[idx]=int(str(average_df.iloc[idx].name.isocalendar()[0])+str(average_df.iloc[idx].name.isocalendar()[1]))
  
  for idx in range(len(average_df)-1):
    average_df['week'].iloc[idx]=average_df['week'].iloc[idx+1]

  if str(average_df['week'].iloc[-1])[-2:]!='52':
    average_df['week'].iloc[-1]=average_df['week'].iloc[-1]+1

  else:
    average_df['week'].iloc[-1]= str(int(str(average_df['week'].iloc[-1])[:4])+1)+'1'

  average_df=average_df.reset_index()
  average_df.index=average_df.week
  average_df=average_df.drop('week',axis=1)

  main_gtrends_df['week']=0
  for idx in range(len(main_gtrends_df)):
    main_gtrends_df['week'].iloc[idx]=int(str(main_gtrends_df.iloc[idx].name.isocalendar()[0])+str(main_gtrends_df.iloc[idx].name.isocalendar()[1]))
  
  for idx in range(len(main_gtrends_df)-1):
    if main_gtrends_df.iloc[idx].name.dayofweek>=4:
      if str(main_gtrends_df['week'].iloc[idx])[-2:]!='52':
        main_gtrends_df['week'].iloc[idx]=main_gtrends_df['week'].iloc[idx]+1

      else:
        main_gtrends_df['week'].iloc[idx]= int(str(int(str(main_gtrends_df['week'].iloc[idx])[:4])+1)+'1')
    
  main_gtrends_df=main_gtrends_df.reset_index()

  main_gtrends_df.index=main_gtrends_df.week
  main_gtrends_df=main_gtrends_df.drop('week',axis=1)

  main_gtrends_df=pd.merge_asof(main_gtrends_df.sort_index(),average_df.sort_index(),right_index=True,left_index=True)

  main_gtrends_df=main_gtrends_df.rename(columns={'date_x':'date'})
  main_gtrends_df.index=main_gtrends_df['date']
  main_gtrends_df=main_gtrends_df.drop(['date_y','date'],axis=1)
  main_gtrends_df=main_gtrends_df.sort_index()

  return main_gtrends_df

def UpdateUniGTrends():
  main_gtrends_df=pd.read_csv('drive/My Drive/Nick Fury/External Datasets/gtrends.csv/gtrends.csv')

  df_date = datetime.strptime(main_gtrends_df['date'].iloc[-1], "%Y-%m-%d").date()
  current_date = datetime.today().date()
  
  if True in main_gtrends_df.columns.str.contains('date'):
    main_gtrends_df.index=main_gtrends_df.date
    main_gtrends_df=main_gtrends_df.iloc[:,1:]
    main_gtrends_df.index=pd.to_datetime(main_gtrends_df.index)

  if current_date > df_date:
    
    if True in main_gtrends_df.columns.str.contains('date'):
      main_gtrends_df.index=main_gtrends_df.date
      main_gtrends_df=main_gtrends_df.iloc[:,1:]
      main_gtrends_df.index=pd.to_datetime(main_gtrends_df.index)

    collector=[]
    keywords_list=main_gtrends_df.columns.tolist()
    timeframe='now 7-d'

    pytrends_object = pytrends.request.TrendReq(hl='en-US', tz=360)

    for keyword in keywords_list:
      
      pytrends_object.build_payload([keyword],timeframe=timeframe)

      single_gtrends_df=pytrends_object.interest_over_time()[keyword]

      single_gtrends_df=single_gtrends_df.resample('1D').mean()

      collector.append(single_gtrends_df)

    update_gtrends_df=pd.concat(collector,axis=1)

    main_gtrends_df=pd.concat([main_gtrends_df,update_gtrends_df])
    main_gtrends_df = main_gtrends_df[~main_gtrends_df.index.duplicated(keep='first')]


    main_gtrends_df.to_csv('drive/My Drive/Nick Fury/External Datasets/gtrends.csv/gtrends.csv')

  return main_gtrends_df

def get_returns(ticker):
  data=yf.Ticker(ticker).history(period='max')
  data=data.drop(['Dividends','Stock Splits'],axis=1)

  data[ticker+'_returns']=data.Close.pct_change(periods=1)*100
  
  fracdev=FracdiffStat()
  data[ticker+'_diff']=fracdev.fit_transform(data.Close.values.reshape(data.Close.shape+(1,)))

  return data[ticker+'_diff']#data[[ticker+'_returns',ticker+'_diff']] 

def Prophesize(data_column):

  proph_data=data_column.reset_index()
  proph_data.columns=['ds','y']

  prophet = Prophet(seasonality_mode='multiplicative')
    
  prophet.fit(proph_data)

  tenet = pd.DataFrame(np.nan,columns=['y'],index=data_column.index)
  tenet['ds']=data_column.index

  tenet['y'].iloc[0]=proph_data['y'].iloc[0]

  t=[1+t*.000545 for t in range(len(tenet))]
  tenet['t']=t

  scaler=MinMaxScaler()
  scaled=scaler.fit_transform(data_column.values.reshape(data_column.values.shape + (1,)))
  tenet['y_scaled']=np.nan
  tenet['y_scaled'].iloc[0]=scaled[0]

  prophecy=prophet.predict(tenet)

  return prophecy

def t2v_prof(data_wo_prof):

  ###time2vec

  t2v_wo_prices=pd.concat([pd.DataFrame(t2v_func(data_wo_prof[col]),columns=[col+'_linear',col+'_periodic']) for col in data_wo_prof.drop(['high_returns','low_returns', 'open_returns', 'close_returns'],axis=1)],axis=1)
  t2v_wo_prices.index=data_wo_prof.index

  t2v_w_prices=pd.DataFrame(t2v_func(data_wo_prof.loc[:,['high_returns','low_returns', 'open_returns', 'close_returns']]),index=data_wo_prof.index,columns=['prices_lin','prices_per'])

 ###matrix profile
  NoVol_df=data_wo_prof[[col for col in data_wo_prof if (col[-6:]!='volume')]]

  m=3

  NoVolProf,_= stumpy.mstump(NoVol_df.T,m=m,discords=True)
  
  NoVolProf_df=pd.DataFrame(NoVolProf,index=NoVol_df.index,columns=NoVol_df.columns[m-1:]+'_prof')

  ###semantic segmentation
  returns_prof=stumpy.stump(data_wo_prof['close_returns'],m=m)

  price_prof=stumpy.stump(data_wo_prof['close'],m=m)

  diff_prof=stumpy.stump(data_wo_prof['close_diff'],m=m)

  returns_cac, regime_locations = stumpy.fluss(returns_prof[:, 1], L=3, n_regimes=2, excl_factor=20)

  price_cac, regime_locations = stumpy.fluss(price_prof[:, 1], L=3, n_regimes=2, excl_factor=20)

  diff_cac, regime_locations = stumpy.fluss(diff_prof[:, 1], L=3, n_regimes=2, excl_factor=20)

  cac=np.vstack((returns_cac,price_cac,diff_cac)).T

  cac_df=pd.DataFrame(cac,index=data_wo_prof['close_returns'].index[m-1:],columns=['returns_cac','price_cac','diff_cac'])

  ###combining everything together

  data=pd.concat([data_wo_prof,NoVolProf_df,t2v_wo_prices,t2v_w_prices,cac_df],axis=1)
    
  data.replace([np.inf, -np.inf], np.nan, inplace=True)
  data=data.dropna()
  
  ###outliers
  outliers=IsolationForest().fit_predict(np.reshape(data['close_returns'].values,(data['close_returns'].shape[0],-1)))
  data['outliers']=outliers
  
  return data

####Main Data

In [ ]:
def Get_Data(main_ticker):
  
  ### sector and asset data + vix
  index_list=['vox','vcr','vdc','vde','vfh','vht','vis','vgt','vaw','vnq','vpu','dbp','vglt','vgit','vgsh','eem','uso','pdbc','gld','vwo','lqd']
  returns_list=[get_returns(index) for index in index_list]

  returns_df=pd.concat(returns_list,axis=1)       #different sectors and asset allocations 

  vix=yf.Ticker('^vix').history(period='max')['Close']
  vix_df=pd.DataFrame(boxcox(vix)[0],columns=['vix'],index=vix.index) #vix

  fracdev=FracdiffStat()

  vix_diff=fracdev.fit_transform(vix.values.reshape(vix.shape+(1,)))
  vix_diff_df=pd.DataFrame(vix_diff,index=vix.index,columns=['vix_diff'])

  returns_df=pd.merge_asof(returns_df,vix_df,left_index=True,right_index=True)
  returns_df=pd.merge_asof(returns_df,vix_diff_df,left_index=True,right_index=True)

  ###individual stock data
  prices=yf.Ticker(main_ticker).history(period='max').dropna()
  prices.columns=[col.lower() for col in prices.columns]
  data=prices.pct_change(periods=1)*100
  data=data.drop(['dividends','stock splits'],axis=1)
  
  data.columns=[column.lower()+'_returns' for column in data.columns]
  data['volume']=boxcox(yf.Ticker(main_ticker).history(period='max')['Volume'].dropna()+0.0000000001)[0]
  data=data.drop(['volume_returns'],axis=1)
  data['mfi']=ta.volume.money_flow_index(high=prices['high'],low=prices['low'],close=prices['close'],volume=prices['volume'])
  data['cci']=ta.trend.cci(high=prices['high'],low=prices['low'],close=prices['close'],window=14)
  data['high_bb2']=ta.volatility.bollinger_hband(close=prices['close'],window=14,window_dev=2)
  data['low_bb2']=ta.volatility.bollinger_lband(close=prices['close'],window=14,window_dev=2)
  data['ema200']=ta.trend.ema_indicator(close=prices['close'],window=200)
  data['ema50']=ta.trend.ema_indicator(close=prices['close'],window=50)
  data['ema20']=ta.trend.ema_indicator(close=prices['close'],window=20)
  data['ema50']=ta.trend.ema_indicator(close=prices['close'],window=10)
  data['rsi']=ta.momentum.rsi(close=prices['close'],window=14)
  data['close_diff']=fracdev.fit_transform(prices.close.dropna().values.reshape(prices.close.dropna().values.shape+(1,)))
  
  individual_data=data

  ###quandl macroeconomic data
  quandl.ApiConfig.api_key = '**************'

  yale_list=['YALE/US_CONF_INDEX_VAL_INDIV','YALE/US_CONF_INDEX_VAL_INST','YALE/US_CONF_INDEX_CRASH_INDIV','YALE/US_CONF_INDEX_CRASH_INST']

  yale_df_list=[quandl.get(code, start_date='2010-1-1', end_date=str(datetime.today().date()))['Index Value'] for code in yale_list] 

  yale_df=pd.DataFrame(yale_df_list,index=yale_list).T  #yale confidence data

  employ_df=quandl.get('FRED/NROUST', start_date='2010-1-1', end_date=str(datetime.today().date())) #short term unemployment]
  product_df=quandl.get('FRED/GDPPOT', start_date='2010-1-1', end_date=str(datetime.today().date())) #nominal gross domestic product

  fed_df=pd.concat([product_df,employ_df],axis=1)
  fed_df.columns=['FRED/NROUST','FRED/GDPPOT'] #federal reserve data

  #zillow real estate_data
  region_ids=['9840', '9980', '9859', '9602', '99886', '99109']

  estate_list=[]
  for id in region_ids:
    single_region=quandl.get_table('ZILLOW/DATA', indicator_id='ZALL', region_id=id)
    single_region.index=single_region.date
    single_region=single_region.drop(['region_id','indicator_id','date'],axis=1)
    estate_list.append(single_region)

  estate_df=pd.concat(estate_list,axis=1)
  estate_df.columns=region_ids

  #aluminum data
  alu_df= quandl.get('WORLDAL/PALPROD', start_date='2010-1-1', end_date=str(datetime.today().date()))[['Total','Daily Average']]
  alu_df.columns=alu_df.columns+'_alu'

  #social security data
  ssn_df=quandl.get('SOCSEC/ALL', start_date='2010-1-1', end_date=str(datetime.today().date()))
  ssn_df.columns=ssn_df.columns+'_ssn'

  #inflation
  infl_df=quandl.get('RATEINF/CPI_USA', start_date='2010-1-1', end_date=str(datetime.today().date()))
  infl_df.columns=['basket_price']
  infl_df['inflation_Q']=infl_df['basket_price'].pct_change()

  #sunspots
  suns_df=quandl.get('SIDC/SUNSPOTS_M', start_date='2010-1-1', end_date=str(datetime.today().date()))['Number of Observations']
  suns_df.columns=['n_sunspots']

  #metals 
  metals_list=['JOHNMATT/RUTH','JOHNMATT/IRID','JOHNMATT/RHOD','JOHNMATT/PALL','JOHNMATT/PLAT']

  single_metals=[quandl.get(metal, start_date='2010-1-1', end_date=str(datetime.today().date())).iloc[:,0] for metal in metals_list]

  metals_df=pd.concat(single_metals,axis=1)
  metals_df.columns=[metal[-4:] for metal in metals_list]
  metals_df=metals_df.fillna(method='ffill')

  for column in metals_df.columns:
    metals_df[column+'_diff']=fracdev.fit_transform(metals_df[column].values.reshape(metals_df[column].shape+(1,)))

  #shortint
  short_df=quandl.get('FINRA/FNSQ_'+main_ticker.upper(), start_date='2010-1-1', end_date=str(datetime.today().date()))['ShortVolume']
  short_df.columns=['short_int']

  ##weekly sentiment
 
  sentiment_df=quandl.get('AAII/AAII_SENTIMENT-AAII-Investor-Sentiment-Data')

  sentiment_df=sentiment_df.dropna()
  sentiment_df=sentiment_df[['Bullish','Neutral','Bearish']].iloc[1:,:]

  ###pytrends
  ticker_gtrends_df=UpdateTickerGTrends(main_ticker)
  ticker_gtrends_df=Add_Weekend(ticker_gtrends_df)
  main_gtrends_df=UpdateUniGTrends()
  main_gtrends_df=Add_Weekend(main_gtrends_df)
  
  #impute nans
  df_list=[short_df,metals_df,suns_df,infl_df,ssn_df,alu_df,estate_df,sentiment_df,fed_df,yale_df]
  data_wo_prof=pd.concat([returns_df,individual_data,prices.close,ticker_gtrends_df],axis=1)

  #imputed_df=impyute.imputation.cs.em(data_wo_prof.values,loops=500)
  #imputed_df=pd.DataFrame(imputed_df)
  #imputed_df.columns=data_wo_prof.columns
  #imputed_df.index=data_wo_prof.index
  for df in df_list:
    data_wo_prof=pd.merge_asof(data_wo_prof,df,right_index=True,left_index=True).dropna()
  
  data_wo_prof=data_wo_prof.loc[:,(data_wo_prof != data_wo_prof.iloc[0]).any()] 
  data_wo_prof = data_wo_prof.loc[:,data_wo_prof.apply(pd.Series.nunique) != 1]

  data=t2v_prof(data_wo_prof).dropna()

  data=pd.concat([data,main_gtrends_df],axis=1)
  
  
  data['label']=0
  data['label'].iloc[np.where(data['close_returns']>0)]=1
  data['label'].iloc[np.where(data['close_returns']<-0)]=-1
  #data['label'].iloc[np.where((data['close_returns']<0) & (data['close_returns']>-0.3))]=0
  data['label'].iloc[np.where(data['close_returns']==0)]=0

  data=data.drop(list(data.index[np.where(data['label']==0)[0]])) 

  scaler=MinMaxScaler((-1,1))

  scaled_data=scaler.fit_transform(data)
  scaled_df=pd.DataFrame(scaled_data,index=data.index,columns=data.columns)
  scaled_df['label']=data['label']

  return scaled_df,data

####Time2Vec

In [ ]:
class Time2Vec(nn.Module):
  def __init__(self, activation,hiddem_dim,in_features):
      super().__init__()
      self.in_features=in_features
      if activation == "sin":
          self.l1 = SineActivation(self.in_features, hiddem_dim)
      elif activation == "cos":
          self.l1 = CosineActivation(self.in_features, hiddem_dim)
      
      self.fc1 = nn.Linear(hiddem_dim, 2)
  
  def forward(self, x):
      #x = x.unsqueeze(1)
      x = self.l1(x)
      x = self.fc1(x)
      return x

def t2v(tau, f, out_features, w, b, w0, b0, arg=None):
    if arg:
        v1 = f(torch.matmul(tau, w) + b, arg)
    else:
        
        v1 = f(torch.matmul(tau, w) + b)
    v2 = torch.matmul(tau, w0) + b0
   
    return torch.cat([v1, v2], 1)

class SineActivation(nn.Module):
    def __init__(self, in_features, out_features):
        super(SineActivation, self).__init__()
        self.out_features = out_features
        self.w0 = nn.parameter.Parameter(torch.randn(in_features, 1))
        self.b0 = nn.parameter.Parameter(torch.randn(1, 1))
        self.w = nn.parameter.Parameter(torch.randn(in_features, out_features-1))
        self.b = nn.parameter.Parameter(torch.randn(1, out_features-1))
        self.f = torch.sin

    def forward(self, tau):
        return t2v(tau, self.f, self.out_features, self.w, self.b, self.w0, self.b0)

def t2v_func(data):
  if len(data.shape) >1:
    time2vec=Time2Vec(activation='sin',hiddem_dim=100,in_features=data.shape[-1])
    t2v_data=time2vec(torch.Tensor(data.values))

  else:
    data=np.reshape(data.values,(data.shape[0],1))
    time2vec=Time2Vec(activation='sin',hiddem_dim=100,in_features=data.shape[-1])
    t2v_data=time2vec(torch.Tensor(data))
  
  if min(t2v_data[:,-1])<=0:
    t2v_data[:,-1]=torch.Tensor(boxcox(((t2v_data[:,-1]+abs(min(t2v_data[:,-1]))+0.00000001)+0.000001).detach())[0])
  
  if min(t2v_data[:,0])<=0:
    t2v_data[:,0]=torch.Tensor(boxcox(((t2v_data[:,0]+abs(min(t2v_data[:,0]))+0.00000001)+0.000001).detach())[0])
  
  return t2v_data.detach().numpy()

###Manual Models

####Bayesian-LSTM

In [ ]:
#defining the neural net
@variational_estimator
class net(nn.Module):
    def __init__(self,hidden_size,dropout,train_inputs,single_label=False):#batch_first=True
        super(net,self).__init__()
        self.hidden_size=hidden_size

        self.input_size=train_inputs.shape[-1] #number of features
        
        self.output_size= self.input_size #should = input_size, but change to 1 if we're interested in only 1 feature

        if single_label==True:
          self.output_size=1

        """self.lstm = torch.nn.LSTM(
            input_size=self.input_size,
            hidden_size=self.hidden_size,
            num_layers=1,
            batch_first=True,
            bias=True,
            dropout=dropout,
            bidirectional=True,
        )"""

        self.dropout=nn.Dropout(dropout)
        self.lstm=BayesianLSTM(self.input_size,self.hidden_size)
        self.linear=BayesianLinear(self.hidden_size,self.output_size)
        #self.linear=nn.Linear(self.hidden_size,self.output_size)
  
    def forward(self, inputs):
        
        output, (hidden,cell) =self.lstm(inputs)

        output=self.dropout(output)
      
        output=self.linear(output)
        
        output=torch.tanh(output)[:,-1,-1]
        
        return output

####Global KNN

In [ ]:
class GlobalWeightedKNN:
  """
  A k-NN classifier with feature weights

  Returns: predictions of k-NN.
  """

  def __init__(self):
      self.X_train = None
      self.y_train = None
      self.k = None
      self.weights = None
      self.tree = None
      self.predictions = list()

  def fit(self, X_train, y_train, k, weights):        
      self.X_train = X_train
      self.y_train = y_train
      self.k = k
      self.weights = weights
      self.tree = BallTree(X_train,metric=DistanceMetric.get_metric('wminkowski', p=2, w=weights))


  def predict(self, testing_data):
      """
      Takes a 2d array of query cases.

      Returns a list of predictions for k-NN classifier
      """
      indexes = self.tree.query(testing_data, self.k, return_distance=False)
      y_answers = self.y_train[indexes]
      self.predictions = np.apply_along_axis(lambda x: mode(x)[0], 1, y_answers)
      return self.predictions

###Splitting Data

In [ ]:
def Tensify(data,train_ratio,sample_len,close_ind,kpca=False,pca=False):
    tensor_list=[]
    
    train_inputs, train_targets, test_inputs, test_targets = Extract_XY(data,train_ratio,sample_len,close_ind,pca,kpca)
    
    data_list=[train_inputs,train_targets,test_inputs,test_targets]
    
    for data in data_list:
        tensor_list.append(torch.Tensor(data)) 
        
    train_inputs=tensor_list[0]
    train_targets=tensor_list[1]
    test_inputs=tensor_list[2]
    test_targets=tensor_list[3]
    
    return train_inputs.to(device),train_targets.to(device),test_inputs.to(device),test_targets.to(device)

def Extract_XY(data,train_ratio,sample_len,close_ind,pca=False,kpca=False):
  chunk_list=[]

  num_targets=len(data)-sample_len

  for timestep in range(num_targets):
      end_index=timestep+sample_len+1 #targets are only '1' index ahead of inputs
      chunk_list.append(data.iloc[timestep:end_index,:].values)

  chunks=np.array(chunk_list)

  to_pca=data.iloc[:round(len(data)*train_ratio)]

  train_data=chunks[:round(train_ratio*data.shape[0])]
  test_data=chunks[round(train_ratio*data.shape[0]):]

  train_inputs=train_data[:,:-1,:]
  #train_targets=train_data[:,1:,:]
  
  train_targets=train_data[:,-1,close_ind] #if we only want to get a single label on the last day
  #train_targets=train_data[:,1:,-1]  #if we want to predict a single feature over next sample_len
  #train_targets=train_data[:,-1,:] #if we want to predict just the last day
  #train_targets=np.reshape(train_targets,(train_targets.shape[0],train_targets.shape[1],-1))
  #train_targets=np.reshape(train_targets,(train_targets.shape[0],-1,train_targets.shape[1]))
  
  
  test_inputs=test_data[:][:,:-1,:]
  #test_targets=test_data[:,1:,:]
  
  test_targets=test_data[:,-1,close_ind]
  #test_targets=test_data[:,1:,-1]
  #test_targets=test_data[:,-1,:] 
  #test_targets=np.reshape(test_targets,(test_targets.shape[0],test_targets.shape[1],-1))
  #test_targets=np.reshape(test_targets,(test_targets.shape[0],-1,test_targets.shape[1]))

  if pca==True:
    pca=PCA(n_components=0.99)
    pca.fit(to_pca)
    train_inputs=np.array([pca.transform(chunk) for chunk in train_inputs])
    test_inputs=np.array([pca.transform(chunk) for chunk in test_inputs])
  
  if kpca==True:
    pca=PCA(n_components=0.99)
    pca.fit(to_pca)
    pca_inputs=np.array([pca.transform(chunk) for chunk in train_inputs])
    kpca=KernelPCA(n_components=pca_inputs.shape[-1]+5,kernel='cosine',fit_inverse_transform=True)
    kpca.fit(to_pca)
    train_inputs=np.array([kpca.transform(chunk) for chunk in train_inputs])
    test_inputs=np.array([kpca.transform(chunk) for chunk in test_inputs])

  return train_inputs,train_targets, test_inputs,test_targets

def ShiftOneLabelSplit(data,train_ratio,column,val=False,include_label=True): #column

  if val==True:
    split_data=data[:round(0.8*len(data))]
    
    train_data=split_data[:round(0.65*len(split_data))]
    val_data=split_data[round(0.35*len(split_data)):]
    test_data=data[round(0.8*len(data)):]
    
    column_ind=data.columns.get_loc(column)
    
    train_inputs=(train_data.iloc[:-1].values)
    train_labels=(train_data.iloc[1:,column_ind].values)
    val_inputs=(val_data.iloc[:-1].values)
    val_labels=(val_data.iloc[1:,column_ind].values)
    test_inputs=(test_data.iloc[:-1].values)
    test_labels=(test_data.iloc[1:,column_ind].values)

    if include_label==False:
      train_inputs=train_inputs[:,:-1]
      val_inputs=val_inputs[:,:-1]
      test_inputs=test_inputs[:,:-1]

    return train_inputs,train_labels,val_inputs,val_labels,test_inputs,test_labels
  
  else: 
    train_data=data[:round(0.8*len(data))]
    test_data=data[round(0.8*len(data)):]

    column_ind=data.columns.get_loc(column)
    
    train_inputs=(train_data.iloc[:-1].values)
    train_labels=(train_data.iloc[1:,column_ind].values)
    test_inputs=(test_data.iloc[:-1].values)
    test_labels=(test_data.iloc[1:,column_ind].values)

    if include_label==False:
      train_inputs=train_inputs[:,:-1]
      test_inputs=test_inputs[:,:-1]

  return train_inputs,train_labels,test_inputs,test_labels

###Feature Importance

In [ ]:
def feat_permute(clf,train_inputs,train_labels,inputs):

  clf.fit(train_inputs,train_labels)
  results=pimp(clf,train_inputs,train_labels,scoring='f1_macro',n_repeats=10)
  perms=pd.Series(abs(results['importances_mean']),index=inputs.columns)
 
  return perms


def feat_importance(train_inputs,train_labels,inputs):

  randf=RandomForestClassifier()
  randf_perm=feat_permute(randf,train_inputs,train_labels,inputs)

  sgc=SGC(loss='log',penalty='l1')
  sgc_perm=feat_permute(sgc,train_inputs,train_labels,inputs)

  nnet=TimeSeriesMLPClassifier(activation='relu',max_iter=8000,alpha=0.0001)
  nnet_perm=feat_permute(nnet,train_inputs,train_labels,inputs)
 
  logreg=LogisticRegression(multi_class='multinomial',penalty='l1',solver='saga',max_iter=300)
  logreg_perm=feat_permute(logreg,train_inputs,train_labels,inputs)

  gbc=GradientBoostingClassifier()
  gbc_perm=feat_permute(gbc,train_inputs,train_labels,inputs)

  ada=AdaBoostClassifier()
  ada_perm=feat_permute(ada,train_inputs,train_labels,inputs)

  gnb=GaussianNB()
  gnb_perm=feat_permute(gnb,train_inputs,train_labels,inputs)

  bag=BaggingClassifier()
  bag_perm=feat_permute(bag,train_inputs,train_labels,inputs)
  
  params_lightGB = {
    'task': 'train',
    'application':'binary',
    'num_class':1,
    'boosting': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'metric_freq':50,
    'is_training_metric':False,
    'max_depth':4,
    'num_leaves': 31,
    'learning_rate': 0.01,
    'feature_fraction': 1.0,
    'bagging_fraction': 1.0,
    'bagging_freq': 0,
    'bagging_seed': 2018,
    'verbose': 0,
    'num_threads':16
    }

  lgb_train=lgb.Dataset(train_inputs[:round(0.8*len(train_inputs))],train_labels[:round(0.8*len(train_inputs))])
  lgb_val=lgb.Dataset(train_inputs[round(0.8*len(train_inputs)):],train_labels[round(0.8*len(train_inputs)):])
  lgbm = lgb.train(params_lightGB, lgb_train,
                      num_boost_round=2000,
                      valid_sets= lgb_val,
                      early_stopping_rounds=200,
                      verbose_eval=False)

  lgb_feat=pd.Series(lgbm.feature_importance(),index=data.columns,name='lgb')

  feat_importance=pd.concat([logreg_perm,sgc_perm,ada_perm,gbc_perm,gnb_perm,bag_perm,nnet_perm,randf_perm,lgb_feat],axis=1)
  scaler=MinMaxScaler((-1,1))
  scaled_feats=scaler.fit_transform(feat_importance)

  scaled_feats_df=pd.DataFrame(scaled_feats,columns=feat_importance.columns,index=feat_importance.index)
  sorted_feats=(abs(scaled_feats_df)).sum(1).sort_values()

  return sorted_feats

###Ensemble

In [ ]:
def StackPred(train_inputs,train_labels,inputs,kpca=False,pca=False):
  orig_inputs=inputs.copy()

  if kpca==True:
    pca=PCA(n_components=0.9999)
    pca_inputs=pca.fit_transform(train_inputs)
    kpca=KernelPCA(n_components=pca_inputs.shape[-1]+5,kernel='cosine',fit_inverse_transform=True)
    train_inputs=kpca.fit_transform(train_inputs)
    print(train_inputs.shape)
    inputs=kpca.transform(inputs)
      
  if pca ==True:
    pca=PCA(n_components=0.9999)
    train_inputs=pca.fit_transform(train_inputs)
    print(train_inputs.shape)
    inputs=pca.transform(inputs)

  gknn=GlobalWeightedKNN()
  gknn.fit(train_inputs,train_labels,k=1,weights=np.random.rand(train_inputs.shape[-1]))
  knn_preds=pd.Series(np.squeeze(gknn.predict(inputs),axis=1),name='knn')

  nnet=TimeSeriesMLPClassifier(activation='relu',max_iter=8000,alpha=0.0001)
  nnet.fit(train_inputs,train_labels)
  nn_preds=pd.Series(nnet.predict(inputs),name='nn')
  nn_probs=pd.Series(nnet.predict_proba(inputs)[:,-1],name='nn')

  svm=TimeSeriesSVC(kernel='rbf',probability=True)
  svm.fit(train_inputs,train_labels)
  svm_preds=pd.Series(svm.predict(inputs),name='svm')
  svm_probs=pd.Series(svm.predict_proba(inputs)[:,-1],name='svm')

  randf=RandomForestClassifier()
  randf.fit(train_inputs,train_labels)
  randf_preds=pd.Series(randf.predict(inputs),name='randf')
  randf_probs=pd.Series(randf.predict_proba(inputs)[:,-1],name='randf')

  sgc=SGC(loss='log',penalty='l1')
  sgc.fit(train_inputs,train_labels)
  sgc_preds=pd.Series(sgc.predict(inputs),name='sgc')
  sgc_probs=pd.Series(sgc.predict_proba(inputs)[:,-1],name='sgc')

  logreg=LogisticRegression(multi_class='multinomial',penalty='l1',solver='saga',max_iter=300)
  logreg.fit(train_inputs,train_labels)
  logreg_preds=pd.Series(logreg.predict(inputs),name='logreg')
  logreg_probs=pd.Series(logreg.predict_proba(inputs)[:,-1],name='logreg')
  
  gnb=GaussianNB()
  gnb.fit(train_inputs,train_labels)
  gnb_preds=pd.Series(gnb.predict(inputs),name='gnb')
  gnb_probs=pd.Series(gnb.predict_proba(inputs)[:,-1],name='gnb')

  gbc=GradientBoostingClassifier()
  gbc.fit(train_inputs,train_labels)
  gbc_preds=pd.Series(gbc.predict(inputs),name='gbc')
  gbc_probs=pd.Series(gbc.predict_proba(inputs)[:,-1],name='gbc')

  ada=AdaBoostClassifier()
  ada.fit(train_inputs,train_labels)
  ada_preds=pd.Series(ada.predict(inputs),name='ada')
  ada_probs=pd.Series(ada.predict_proba(inputs)[:,-1],name='ada')

  bag=BaggingClassifier()
  bag.fit(train_inputs,train_labels)
  bag_preds=pd.Series(bag.predict(inputs),name='bag')
  bag_probs=pd.Series(bag.predict_proba(inputs)[:,-1],name='bag')
  
  scaler=MinMaxScaler((-1,1))
  params_lightGB = {
    'task': 'train',
    'application':'binary',
    'num_class':1,
    'boosting': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'metric_freq':50,
    'is_training_metric':False,
    'max_depth':4,
    'num_leaves': 31,
    'learning_rate': 0.01,
    'feature_fraction': 1.0,
    'bagging_fraction': 1.0,
    'bagging_freq': 0,
    'bagging_seed': 2018,
    'verbose': 0,
    'num_threads':16
    }
  
  lgb_train=lgb.Dataset(train_inputs[:round(0.8*len(train_inputs))],train_labels[:round(0.8*len(train_inputs))])
  lgb_val=lgb.Dataset(train_inputs[round(0.8*len(train_inputs)):],train_labels[round(0.8*len(train_inputs)):])
  lgbm = lgb.train(params_lightGB, lgb_train,
                      num_boost_round=2000,
                      valid_sets= lgb_val,
                      early_stopping_rounds=200,
                      verbose_eval=False)
  lgb_preds=pd.Series(lgbm.predict(inputs),name='lgb')

  scaled_lgb_probs=pd.DataFrame(scaler.fit_transform(lgb_preds.values.reshape(lgb_preds.shape+(1,))),columns=['lgb'])
  scaled_lgb_preds=pd.Series([1 if x>0 else -1 if x<0 else 0 for x in scaled_lgb_probs.values],name='lgb')

  prophecy=Prophesize(orig_inputs['close_returns'])
  detrended=signal.detrend(prophecy['yhat'])
  detrended_series=pd.Series(detrended,name='prophet')
  prophet_preds=pd.Series(np.array([1 if x>0 else -1 if x<0 else 0 for x in detrended]),name='prophet')
  
  sample_len=17
  close_ind=orig_inputs.columns.get_loc('label')
  
  train_inputs,train_targets,test_inputs,test_targets=Tensify(orig_inputs,0.7,sample_len,kpca=kpca,pca=pca,close_ind=close_ind)

  model=net(100,0.1,train_inputs,single_label=True).to(device)

  optimizer = torch.optim.Adam(
      lr=0.001,
      amsgrad=True,
      eps=1e-7,
      params=model.parameters(),
      weight_decay=0.001
  )

  loss_func=nn.L1Loss()
  num_epochs=300

  train_losses=[]
  loss_sum=0
  for epoch in range(num_epochs):
    avg_loss,output=train_step(train_inputs,train_targets,model,loss_func,optimizer)
    
    loss_sum+=avg_loss
    if (epoch % 5) == 4: 
        train_losses.append(loss_sum/5)
        loss_sum=0

  scaled_inputs=torch.cat((train_inputs,test_inputs))
  scaled_targets=torch.cat((train_targets,test_targets))
  
  evals, _=eval_step(scaled_inputs,scaled_targets,model,loss_func)
  pad = pd.Series(np.nan,index=list(range(sample_len)),name='lstm')
  lstm_preds=pd.Series(np.array([1 if x>0 else -1 if x<0 else 0 for x in evals]),name='lstm')
  lstm_probs=pd.Series(evals.cpu().detach().numpy(),name='lstm')
  padded_preds=pd.concat([pad,lstm_preds],ignore_index=True)
  padded_probs=pd.concat([pad,lstm_probs],ignore_index=True)

  pred_df=pd.concat([svm_preds,randf_preds,nn_preds,sgc_preds,knn_preds,gbc_preds,logreg_preds,gnb_preds,ada_preds,bag_preds,padded_preds,prophet_preds,scaled_lgb_preds],axis=1) 
  proba_df=pd.concat([svm_probs,knn_preds,randf_probs,nn_probs,sgc_probs,gbc_probs,gnb_probs,logreg_probs,ada_probs,bag_probs,padded_probs,detrended_series,scaled_lgb_probs],axis=1)  

  #pred_df=pd.concat([padded_preds,gnb_preds,sgc_preds,logreg_preds,ada_preds,prophet_preds,scaled_lgb_preds,nn_preds],axis=1)
  #proba_df=pd.concat([padded_probs,gnb_probs,sgc_probs,logreg_probs,ada_probs,detrended_series,scaled_lgb_probs,nn_probs],axis=1)

  proba_df.index=orig_inputs.index
  pred_df.index=orig_inputs.index

  proba_df['label']=orig_inputs['label']
  pred_df['label']=orig_inputs['label']

  return pred_df.dropna(),proba_df.dropna()

####Train, Evaluate, and Predict

In [ ]:
def train_step(inputs, targets,model,loss_func,optimizer):
    
    model.zero_grad()
    output=model(inputs)
    
    loss = loss_func(output,targets)
    
    loss.backward()#(retain_graph=True)
    optimizer.step()
    
    avg_loss=loss
    return avg_loss,output
 
def eval_step(inputs,targets,model,loss_func):
    model.eval()
    evals= model(inputs)
    loss=loss_func(evals,targets)
    avg_loss=loss
    return evals, avg_loss
 
def confidence_test(confidence,evals,test_targets):
 
  evals_list=[]
  target_list=[]
 
  for i in range(len(evals)):
    if evals[i]>confidence:
      evals_list.append(1)
      target_list.append(test_targets[i].item())
 
    elif evals[i]<-confidence:
      evals_list.append(-1)
      target_list.append(test_targets[i].item())
    
    else:
      continue
 
  evals_arr=np.array(evals_list)
  target_arr=np.array(target_list)
 
  f1_score=f1(evals_arr,target_arr,average='macro')
  
  return evals_arr,target_arr,f1_score
 
def Train_and_Evaluate(ensemble_df,train_ratio,sample_len):
  train_inputs,train_targets,test_inputs,test_targets=Tensify(ensemble_df,train_ratio,sample_len,ensemble_df.columns.get_loc('label'))
  
  model=net(100,0.2,train_inputs,single_label=True).to(device)
  
  optimizer = torch.optim.Adam(
      lr=0.001,
      amsgrad=True,
      params=model.parameters(),
      weight_decay=0.001
  )
 
  loss_func= nn.L1Loss()
  
  num_epochs=150
  train_losses=[]
  loss_sum=0
  for epoch in range(num_epochs):
    avg_loss,output=train_step(train_inputs,train_targets,model,loss_func,optimizer)
    
    loss_sum+=avg_loss
    if (epoch % 5) == 4: 
        train_losses.append(loss_sum/5)
        loss_sum=0
 
  plt.plot(train_losses)
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.title('Training Loss')
 
  evals, eval_loss=eval_step(test_inputs,test_targets,model,loss_func)
 
  trans_evals=np.array([1 if x>0 else -1 if x<0 else 0 for x in evals])
  trans_preds=np.array([1 if x>0 else -1 if x<0 else 0 for x in output])
  print('Training f1_score:',f1(train_targets.cpu().numpy(),trans_preds,average='macro'),'for',ticker)
  print('Testing f1_score', f1(test_targets.cpu().numpy(),trans_evals,average='macro'),'for',ticker)
 
  return model,evals,test_targets
 
def Predict(ensemble_df,model,evals,test_targets,sample_len):

  predict_input=torch.Tensor(ensemble_df.iloc[-sample_len:,:].values).unsqueeze(0).to(device)
  prediction=model(predict_input).item()
  print('prediction',prediction)

  evals_arr,target_arr,f1_score=confidence_test(prediction,evals,test_targets)
  print('f1 score for confidence',f1_score)

  seq_len=10
  f1_list=[]
  for i in range(len(target_arr)):

    f1_list.append(f1(target_arr[i:i+seq_len],evals_arr[i:i+seq_len],average='macro'))

  print('last days f1',f1_list[-1])
  
  return f1_list,prediction,f1_score
 
def To_Judge(pred_df,prob_df):
    
    sample_len=5
    pred_model,pred_evals,pred_test_targets=Train_and_Evaluate(pred_df,0.7,sample_len)
    prob_model,prob_evals,prob_test_targets=Train_and_Evaluate(prob_df,0.7,sample_len)
    
    pred_f1,pred_prediction,pred_f1_score=Predict(pred_df,pred_model,pred_evals,pred_test_targets,sample_len)
    if (pred_f1_score>0.7):
      if (pred_f1[-1] >=0.8):
        return pred_prediction*pred_f1_score*pred_f1[-1]
    
    else:
      prob_f1,prob_prediction,prob_f1_score=Predict(prob_df,prob_model,prob_evals,prob_test_targets,sample_len)

      if (prob_f1_score >0.7):
        if (prob_f1[-1] >=0.8):
          return prob_prediction*prob_f1_score*prob_f1[-1]
 
      else:
        return None

####Bet Sizing

In [ ]:
def Prices(stocks_list):

  prices_df=pd.DataFrame()
  
  for stock_symbol in stocks_list:
     
    data=pdr.get_data_yahoo(stock_symbol)
    
    prices_df[str(stock_symbol)]=data['Close']
    
    prices_df=prices_df.dropna(axis=0)

  return prices_df

def Blacklitter(prices_df,view_dict):
  risk_model=pypfopt.risk_models.CovarianceShrinkage(prices_df)
  cov_matrix=risk_model.oracle_approximating()

  view_series=pd.Series(view_dict)

  cov_matrix=pypfopt.risk_models.fix_nonpositive_semidefinite(cov_matrix,fix_method='spectral')

  if np.linalg.det(cov_matrix)==0:
    cov_matrix=cov_matrix+0.0000001
  
  bl=pypfopt.black_litterman.BlackLittermanModel(cov_matrix=cov_matrix,absolute_views=view_series)

  weights=bl.bl_weights()
  performance=bl.portfolio_performance()

  return weights

###Putting It All Together

In [ ]:
class StockPredictor:
  def __init__(self):
    string=input()
    self.ticker_list=string.split(',')
  
  def Do_Everything(self):
    views_dict={}
    for main_ticker in self.ticker_list:
      #try:
        global ticker
        ticker=main_ticker
        print('--'*4,ticker,'--'*4)
        scaled_df,data=Get_Data(ticker)
        training_inputs,training_labels,test_inputs,test_labels=ShiftOneLabelSplit(scaled_df,0.7,'label',val=False)
        pred_df,prob_df=StackPred(training_inputs,training_labels,scaled_df,pca=True)
        
        pca_prediction=To_Judge(pred_df,prob_df)
    
        if pca_prediction==None:
          pred_df,prob_df=StackPred(training_inputs,training_labels,scaled_df)
          reg_prediction=To_Judge(pred_df,prob_df)

          if reg_prediction!=None:
            views_dict[ticker]=reg_prediction
            
          else:
            pred_df,prob_df=StackPred(training_inputs,training_labels,scaled_df,kpca=True)
            kpca_prediction=To_Judge(pred_df,prob_df)
            if kpca_prediction!=None:
              views_dict[ticker]=kpca_prediction

        else:
          views_dict[ticker]=pca_prediction 
      
      #except:
        #pass
    
    prices_df=Prices(views_dict.keys())
    weights=Blacklitter(prices_df,views_dict)

    for key,value in weights.items():
      data=pdr.get_data_yahoo(key)
      price=data.Close[-1]
      shares=value/price
      print('Get',shares,'shares of',key)

    return weights

In [ ]:
stockpred=StockPredictor()
weights=stockpred.Do_Everything()

In [ ]:
def TickerTrends_Daily(ticker):
  keywords=['buy '+ticker+' stock','sell '+ticker+' stock',ticker+' earnings'] #ticker+' stock'

  df=yf.Ticker(ticker).history(period='max')

  collector=[]

  if df.index[0].year >2010:
    for search_term in keywords:
      try:
    
        daily_data=pytrends.dailydata.get_daily_data(search_term,start_year=df.index[0].year,start_mon=df.index[0].month,stop_year=datetime.now().date().year,stop_mon=datetime.now().date().month)

        data=daily_data[search_term+'_unscaled']
        data=data.rename(search_term)
        collector.append(data)

      except:
        try:
          search_term=search_term[:-6]
          daily_data=pytrends.dailydata.get_daily_data(search_term,start_year=df.index[0].year,start_mon=df.index[0].month,stop_year=datetime.now().date().year,stop_mon=datetime.now().date().month)

          data=daily_data[search_term+'_unscaled']
          data=data.rename(search_term)
          collector.append(data)

        except:
          pass
  
  
  else:
    for search_term in keywords:
      try:
        daily_data=pytrends.dailydata.get_daily_data(search_term,start_year=2010,start_mon=1,stop_year=datetime.now().date().year,stop_mon=datetime.now().date().month)

        data=daily_data[search_term+'_unscaled']
        data=data.rename(search_term)
        collector.append(data)

      except:
        try:
          search_term=search_term[:-6]
          daily_data=pytrends.dailydata.get_daily_data(search_term,start_year=2010,start_mon=1,stop_year=datetime.now().date().year,stop_mon=datetime.now().date().month)

          data=daily_data[search_term+'_unscaled']
          data=data.rename(search_term)
          collector.append(data)

        except:
          pass

  ticker_data=pd.concat(collector,axis=1)
    
  return ticker_data

def CombineTickerTrends(ticker):
  ticker_gtrends_df=UpdateTickerGTrends(ticker)

  other_df=TickerTrends_Daily(ticker)

  ticker_gtrends_df=pd.concat([ticker_gtrends_df,other_df],axis=1).dropna()

  ticker_gtrends_df.to_csv('drive/My Drive/Nick Fury/External Datasets/gtrends.csv/{}.csv'.format(ticker))

  return ticker_gtrends_df

In [ ]:
def UpdateTickerGTrends(ticker):
  ticker_gtrends_df=pd.read_csv('drive/My Drive/Nick Fury/External Datasets/gtrends.csv/{}.csv'.format(ticker)).dropna()

  ticker_gtrends_df.index=ticker_gtrends_df.date
  ticker_gtrends_df=ticker_gtrends_df.iloc[:,1:]
  ticker_gtrends_df.index=pd.to_datetime(ticker_gtrends_df.index)

  timeframe='now 7-d'

  collector=[]

  for keyword in ticker_gtrends_df.columns:

    pytrends_object = pytrends.request.TrendReq(hl='en-US', tz=360)

    pytrends_object.build_payload([keyword],timeframe=timeframe)

    single_gtrends_df=pytrends_object.interest_over_time()[keyword]

    single_gtrends_df=single_gtrends_df.resample('1D').mean()
    
    collector.append(single_gtrends_df)

  multi_gtrends_df=pd.concat(collector,axis=1)

  ticker_gtrends_df=pd.concat([ticker_gtrends_df,multi_gtrends_df])

  ticker_gtrends_df = ticker_gtrends_df[~ticker_gtrends_df.index.duplicated(keep='first')]

  ticker_gtrends_df = ticker_gtrends_df.loc[:,~ticker_gtrends_df.columns.duplicated()]

  ticker_gtrends_df.to_csv('drive/My Drive/Nick Fury/External Datasets/gtrends.csv/{}.csv'.format(ticker))

  return ticker_gtrends_df